In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
import random
import logging
import os
import torch.optim as optim
import torch.distributed as dist
from torch.utils.data import DataLoader
from pathlib import Path



In [2]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class Args:
    nGPU: int = 1
    seed: int = 0
    prepare: bool = True
    mode: str = "train"
    train_data_dir: str = "/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/MINDsmall_train"
    test_data_dir: str = "/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/MINDsmall_dev"
    train_abstract_dir: str = '/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/genAbs0.json'
    # "/content/drive/MyDrive/Colab Notebooks/NewsRecommendation/data/train_gen_abs.json"
    test_abstract_dir: str = '/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/genAbs0.json'
    # "/content/drive/MyDrive/Colab Notebooks/NewsRecommendation/data/Dev_gen_abs.json"
    model_dir: str = '/content/model'
    batch_size: int = 32
    npratio: int = 4
    enable_gpu: bool = False
    filter_num: int = 3
    log_steps: int = 100
    epochs: int = 5
    lr: float = 0.0003
    num_words_title: int = 20
    num_words_abstract: int = 50
    user_log_length: int = 50
    word_embedding_dim: int = 300
    glove_embedding_path: str = '/home/vinmike/Downloads/glove.840B.300d.txt'
    freeze_embedding: bool = False
    news_dim: int = 400
    news_query_vector_dim: int = 200
    user_query_vector_dim: int = 200
    num_attention_heads: int = 15
    user_log_mask: bool = False
    dropout_probability = 0.2
    save_steps: int = 10000
    start_epoch: int = 0
    load_ckpt_name: Optional[str] = None
    use_category: bool = True
    use_subcategory: bool = True
    use_abstract: bool = True
    use_custom_abstract: bool = True
    category_emb_dim: int = 100
    num_filters = 300
    window_size = 3
    long_short_term_method = 'ini'
    masking_probability = 0.5
    num_users = 1 + 50000
    num_categories = 1 + 274

def parse_args():
  return Args()


**Dataset.py**

In [3]:
from torch.utils.data import IterableDataset, Dataset
import numpy as np
import random
import json


class DatasetTrain(IterableDataset):
    def __init__(self, filename, news_index, news_combined, user2id_path, args):
        super(DatasetTrain).__init__()
        self.filename = filename
        self.news_index = news_index  # Maps news ID -> index
        self.news_combined = news_combined  # News feature matrix
        self.args = args

        # Load user2id mapping from file
        with open(user2id_path, "r") as f:
            self.user2id = json.load(f)

    def trans_to_nindex(self, nids):
        return [self.news_index[i] if i in self.news_index else 0 for i in nids]

    def pad_to_fix_len(self, x, fix_length, padding_front=True, padding_value=0):
        if padding_front:
            pad_x = [padding_value] * (fix_length - len(x)) + x[-fix_length:]
            mask = [0] * (fix_length - len(x)) + [1] * min(fix_length, len(x))
        else:
            pad_x = x[-fix_length:] + [padding_value] * (fix_length - len(x))
            mask = [1] * min(fix_length, len(x)) + [0] * (fix_length - len(x))
        return pad_x, np.array(mask, dtype='float32')

    def line_mapper(self, line):
        line = line.strip().split("\t")

        user_id = line[0]  # First column is user ID
        click_docs = line[3].split()  # Clicked news IDs
        sess_pos = line[4].split()  # Positive candidate news IDs
        sess_neg = line[5].split()  # Negative candidate news IDs

        # Convert user ID to index, default to "unk" if missing
        user_index = self.user2id.get(user_id, self.user2id["unk"])

        # Convert clicked news IDs to indices print(f"user shape: {user.shape}")
        
        clicked_news = self.trans_to_nindex(click_docs)
        clicked_news_length = len(clicked_news)
        

        # Pad clicked news to fixed length
        clicked_news, _ = self.pad_to_fix_len(clicked_news, self.args.user_log_length)
        clicked_news_feature = self.news_combined[clicked_news]

        # Sample candidate news (positive + negative)
        label = random.randint(0, self.args.npratio)
        sample_news = sess_neg[:label] + sess_pos + sess_neg[label:]
        sample_news, _ = self.pad_to_fix_len(self.trans_to_nindex(sample_news), self.args.user_log_length)

        candidate_news_feature = self.news_combined[sample_news]
        
        return user_index, clicked_news_length, candidate_news_feature, clicked_news_feature, label

    def __iter__(self):
        file_iter = open(self.filename)
        return map(self.line_mapper, file_iter)



class DatasetTest(DatasetTrain):
    def __init__(self, filename, news_index, news_scoring, args):
        super(DatasetTrain).__init__()
        self.filename = filename
        self.news_index = news_index
        self.news_scoring = news_scoring
        self.args = args

    def line_mapper(self, line):
        line = line.strip().split('\t')
        click_docs = line[3].split()
        click_docs, log_mask = self.pad_to_fix_len(self.trans_to_nindex(click_docs), self.args.user_log_length)
        user_feature = self.news_scoring[click_docs]

        candidate_news = self.trans_to_nindex([i.split('-')[0] for i in line[4].split()])
        labels = np.array([int(i.split('-')[1]) for i in line[4].split()])
        news_feature = self.news_scoring[candidate_news]

        return user_feature, log_mask, news_feature, labels

    def __iter__(self):
        file_iter = open(self.filename)
        return map(self.line_mapper, file_iter)


class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return self.data.shape[0]


**Metric.py**

In [4]:
from sklearn.metrics import roc_auc_score
import numpy as np


def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2**y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


def ctr_score(y_true, y_score, k=1):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    return np.mean(y_true)

def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot



**Ultis.py**

In [5]:
import logging
import argparse
import sys

def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)


def dump_args(args):
    for arg in dir(args):
        if not arg.startswith("_"):
            logging.info(f"args[{arg}]={getattr(args, arg)}")

def load_matrix(embedding_file_path, word_dict, word_embedding_dim):
    embedding_matrix = np.zeros(shape=(len(word_dict) + 1, word_embedding_dim))
    have_word = []
    if embedding_file_path is not None:
        with open(embedding_file_path, 'rb') as f:
            while True:
                line = f.readline()
                if len(line) == 0:
                    break
                line = line.split()
                word = line[0].decode()
                if word in word_dict:
                    index = word_dict[word]
                    tp = [float(x) for x in line[1:]]
                    embedding_matrix[index] = np.array(tp)
                    have_word.append(word)
    return embedding_matrix, have_word


def get_checkpoint(directory, ckpt_name):
    ckpt_path = os.path.join(directory, ckpt_name)
    if os.path.exists(ckpt_path):
        return ckpt_path
    else:
        return None


**preprocess.py**

In [6]:
from collections import Counter
from tqdm import tqdm
import numpy as np
from nltk.tokenize import word_tokenize
import json


def update_dict(dict, key, value=None):
    if key not in dict:
        if value is None:
            dict[key] = len(dict) + 1
        else:
            dict[key] = value


def read_custom_abstract(news_file, custom_abstract_dict):
    news = {}
    news_index = {}
    category_dict = {}
    subcategory_dict = {}
    word_cnt = {}

    with open(news_file, 'r', encoding='utf-8') as f:
        for line in f:
            splited = line.strip('\n').split('\t')
            doc_id, category, subcategory, title, abstract, url, entity_title, entity_abstract = splited
            if doc_id in custom_abstract_dict:
                abstract = custom_abstract_dict[doc_id]
            news[doc_id] = [title.split(' '), category, subcategory, abstract.split(' ')]
            news_index[doc_id] = len(news_index) + 1
            for word in title.split(' '):
                if word not in word_cnt:
                    word_cnt[word] = 0
                word_cnt[word] += 1
            for word in abstract.split(' '):
                if word not in word_cnt:
                    word_cnt[word] = 0
                word_cnt[word] += 1
            if category not in category_dict:
                category_dict[category] = len(category_dict) + 1
            if subcategory not in subcategory_dict:
                subcategory_dict[subcategory] = len(subcategory_dict) + 1

    return news, news_index, category_dict, subcategory_dict, word_cnt

def read_news(news_path, abstract_path, args, mode='train'):
    news = {}
    category_dict = {}
    subcategory_dict = {}
    news_index = {}
    word_cnt = Counter()
    if args.use_custom_abstract:
      with open(abstract_path, 'r') as f:
          abs = json.load(f)
    with open(news_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            splited = line.strip('\n').split('\t')
            doc_id, category, subcategory, title, abstract, url, _, _ = splited
            update_dict(news_index, doc_id)

            title = title.lower()
            title = word_tokenize(title, language='english', preserve_line=True)

            update_dict(news, doc_id, [title, category, subcategory, abs[doc_id] if doc_id in abs else abstract])
            if mode == 'train':
                if args.use_category:
                    update_dict(category_dict, category)
                if args.use_subcategory:
                    update_dict(subcategory_dict, subcategory)
                word_cnt.update(title)

    if mode == 'train':
        word = [k for k, v in word_cnt.items() if v > args.filter_num]
        word_dict = {k: v for k, v in zip(word, range(1, len(word) + 1))}
        return news, news_index, category_dict, subcategory_dict, word_dict
    elif mode == 'test':
        return news, news_index
    else:
        assert False, 'Wrong mode!'


def get_doc_input(news, news_index, category_dict, subcategory_dict, word_dict, args):
    news_num = len(news) + 1
    news_title = np.zeros((news_num, args.num_words_title), dtype='int32')
    news_category = np.zeros((news_num, 1), dtype='int32') if args.use_category else None
    news_subcategory = np.zeros((news_num, 1), dtype='int32') if args.use_subcategory else None
    news_abstract = np.zeros((news_num, args.num_words_abstract), dtype='int32') if args.use_abstract else None

    for key in tqdm(news):
        title, category, subcategory, abstract = news[key]
        doc_index = news_index[key]

        for word_id in range(min(args.num_words_title, len(title))):
            if title[word_id] in word_dict:
                news_title[doc_index, word_id] = word_dict[title[word_id]]

        if args.use_category:
            news_category[doc_index, 0] = category_dict[category] if category in category_dict else 0
        if args.use_subcategory:
            news_subcategory[doc_index, 0] = subcategory_dict[subcategory] if subcategory in subcategory_dict else 0
        if args.use_abstract:
            for word_id in range(min(args.num_words_abstract, len(abstract))):
                if abstract[word_id] in word_dict:
                    news_abstract[doc_index, word_id] = word_dict[abstract[word_id]]

    return news_title, news_category, news_subcategory, news_abstract

**prepare_data.py**

In [7]:
import os
from tqdm import tqdm
import random
import logging


def get_sample(all_elements, num_sample):
    if num_sample > len(all_elements):
        return random.sample(all_elements * (num_sample // len(all_elements) + 1), num_sample)
    else:
        return random.sample(all_elements, num_sample)


def prepare_training_data(train_data_dir, nGPU, npratio, seed):
    random.seed(seed)
    behaviors = []

    behavior_file_path = os.path.join(train_data_dir, 'behaviors.tsv')
    with open(behavior_file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            iid, uid, time, history, imp = line.strip().split('\t')
            impressions = [x.split('-') for x in imp.split(' ')]
            pos, neg = [], []
            for news_ID, label in impressions:
                if label == '0':
                    neg.append(news_ID)
                elif label == '1':
                    pos.append(news_ID)
            if len(pos) == 0 or len(neg) == 0:
                continue
            for pos_id in pos:
                neg_candidate = get_sample(neg, npratio)
                neg_str = ' '.join(neg_candidate)
                new_line = '\t'.join([iid, uid, time, history, pos_id, neg_str]) + '\n'
                behaviors.append(new_line)

    random.shuffle(behaviors)

    behaviors_per_file = [[] for _ in range(nGPU)]
    for i, line in enumerate(behaviors):
        behaviors_per_file[i % nGPU].append(line)

    logging.info('Writing files...')
    for i in range(nGPU):
        processed_file_path = os.path.join(train_data_dir, f'behaviors_np{npratio}_{i}.tsv')
        with open(processed_file_path, 'w') as f:
            f.writelines(behaviors_per_file[i])

    return len(behaviors)


def prepare_testing_data(test_data_dir, nGPU):
    behaviors = [[] for _ in range(nGPU)]

    behavior_file_path = os.path.join(test_data_dir, 'behaviors.tsv')
    with open(behavior_file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(tqdm(f)):
            behaviors[i % nGPU].append(line)

    logging.info('Writing files...')
    for i in range(nGPU):
        processed_file_path = os.path.join(test_data_dir, f'behaviors_{i}.tsv')
        with open(processed_file_path, 'w') as f:
            f.writelines(behaviors[i])

    return sum([len(x) for x in behaviors])


In [8]:
def train(rank, args):

    is_distributed = False
    torch.cuda.set_device(rank)


    news, news_index, category_dict, subcategory_dict, word_dict = read_news(
        os.path.join(args.train_data_dir, 'news.tsv'), args.train_abstract_dir, args, mode='train')

    news_title, news_category, news_subcategory, news_abstract = get_doc_input(
        news, news_index, category_dict, subcategory_dict, word_dict, args)
    news_combined = np.concatenate([x for x in [news_title, news_category, news_subcategory, news_abstract] if x is not None], axis=-1)

    if rank == 0:
        logging.info('Initializing word embedding matrix...')

    embedding_matrix, have_word = load_matrix(args.glove_embedding_path,
                                                    word_dict,
                                                    args.word_embedding_dim)
    if rank == 0:
        logging.info(f'Word dict length: {len(word_dict)}')
        logging.info(f'Have words: {len(have_word)}')
        logging.info(f'Missing rate: {(len(word_dict) - len(have_word)) / len(word_dict)}')

    model = Model(args, embedding_matrix, len(category_dict), len(subcategory_dict))

    if args.load_ckpt_name is not None:
        ckpt_path = get_checkpoint(args.model_dir, args.load_ckpt_name)
        checkpoint = torch.load(ckpt_path, map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'])
        logging.info(f"Model loaded from {ckpt_path}.")

    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.enable_gpu:
        model = model.cuda(rank)

    if is_distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[rank])

    # if rank == 0:
    #     print(model)
    #     for name, param in model.named_parameters():
    #         print(name, param.requires_grad)

    data_file_path = os.path.join(args.train_data_dir, f'behaviors_np{args.npratio}_{rank}.tsv')

    dataset = DatasetTrain(data_file_path, news_index, news_combined, args)
    dataloader = DataLoader(dataset, batch_size=args.batch_size)

    logging.info('Training...')
    for ep in range(args.start_epoch, args.epochs):
        loss = 0.0
        accuary = 0.0
        for cnt, (log_ids, log_mask, input_ids, targets) in enumerate(dataloader):
            if args.enable_gpu:
                log_ids = log_ids.cuda(rank, non_blocking=True)
                log_mask = log_mask.cuda(rank, non_blocking=True)
                input_ids = input_ids.cuda(rank, non_blocking=True)
                targets = targets.cuda(rank, non_blocking=True)

            bz_loss, y_hat = model(log_ids, log_mask, input_ids, targets)
            loss += bz_loss.data.float()
            accuary += acc(targets, y_hat)
            optimizer.zero_grad()
            bz_loss.backward()
            optimizer.step()

            if cnt % args.log_steps == 0:
                logging.info(
                    '[{}] Ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(
                        rank, cnt * args.batch_size, loss.data / cnt, accuary / cnt)
                )

            if rank == 0 and     cnt != 0 and cnt % args.save_steps == 0:
                ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}-{cnt}.pt')
                torch.save(
                    {
                        'model_state_dict':
                            {'.'.join(k.split('.')[1:]): v for k, v in model.state_dict().items()}
                            if is_distributed else model.state_dict(),
                        'category_dict': category_dict,
                        'word_dict': word_dict,
                        'subcategory_dict': subcategory_dict
                    }, ckpt_path)
                logging.info(f"Model saved to {ckpt_path}.")

        logging.info('Training finish.')

        if rank == 0:
            ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}.pt')
            torch.save(
                {
                    'model_state_dict':
                        {'.'.join(k.split('.')[1:]): v for k, v in model.state_dict().items()}
                        if is_distributed else model.state_dict(),
                    'category_dict': category_dict,
                    'subcategory_dict': subcategory_dict,
                    'word_dict': word_dict,
                }, ckpt_path)
            logging.info(f"Model saved to {ckpt_path}.")



In [9]:

    import subprocess
    setuplogger()
    args = parse_args()
    dump_args(args)
    random.seed(args.seed)





[INFO 2025-02-25 00:45:22,237] args[batch_size]=32
[INFO 2025-02-25 00:45:22,238] args[category_emb_dim]=100
[INFO 2025-02-25 00:45:22,239] args[dropout_probability]=0.2
[INFO 2025-02-25 00:45:22,240] args[enable_gpu]=False
[INFO 2025-02-25 00:45:22,241] args[epochs]=5
[INFO 2025-02-25 00:45:22,242] args[filter_num]=3
[INFO 2025-02-25 00:45:22,243] args[freeze_embedding]=False
[INFO 2025-02-25 00:45:22,243] args[glove_embedding_path]=/home/vinmike/Downloads/glove.840B.300d.txt
[INFO 2025-02-25 00:45:22,244] args[load_ckpt_name]=None
[INFO 2025-02-25 00:45:22,245] args[log_steps]=100
[INFO 2025-02-25 00:45:22,246] args[long_short_term_method]=ini
[INFO 2025-02-25 00:45:22,247] args[lr]=0.0003
[INFO 2025-02-25 00:45:22,247] args[masking_probability]=0.5
[INFO 2025-02-25 00:45:22,248] args[mode]=train
[INFO 2025-02-25 00:45:22,249] args[model_dir]=/content/model
[INFO 2025-02-25 00:45:22,252] args[nGPU]=1
[INFO 2025-02-25 00:45:22,252] args[news_dim]=400
[INFO 2025-02-25 00:45:22,253] arg

# **NRMS**

In [10]:
import torch
import torch.nn as nn
class DotProductClickPredictor(torch.nn.Module):
    def __init__(self):
        super(DotProductClickPredictor, self).__init__()

    def forward(self, candidate_news_vector, user_vector):
        """
        Args:
            candidate_news_vector: batch_size, candidate_size, X
            user_vector: batch_size, X
        Returns:
            (shape): batch_size
        """
        # batch_size, candidate_size
        probability = torch.bmm(candidate_news_vector,
                                user_vector.unsqueeze(dim=-1)).squeeze(dim=-1)
        return probability
class AdditiveAttention(torch.nn.Module):
    """
    A general additive attention module.
    Originally for NAML.
    """
    def __init__(self,
                 query_vector_dim,
                 candidate_vector_dim,
                 writer=None,
                 tag=None,
                 names=None):
        super(AdditiveAttention, self).__init__()
        self.linear = nn.Linear(candidate_vector_dim, query_vector_dim)
        self.attention_query_vector = nn.Parameter(
            torch.empty(query_vector_dim).uniform_(-0.1, 0.1))
        # For tensorboard
        self.writer = writer
        self.tag = tag
        self.names = names
        self.local_step = 1

    def forward(self, candidate_vector):
        """
        Args:
            candidate_vector: batch_size, candidate_size, candidate_vector_dim
        Returns:
            (shape) batch_size, candidate_vector_dim
        """
        # batch_size, candidate_size, query_vector_dim
        temp = torch.tanh(self.linear(candidate_vector))
        # batch_size, candidate_size
        candidate_weights = F.softmax(torch.matmul(
            temp, self.attention_query_vector),
                                      dim=1)
        if self.writer is not None:
            assert candidate_weights.size(1) == len(self.names)
            if self.local_step % 10 == 0:
                self.writer.add_scalars(
                    self.tag, {
                        x: y
                        for x, y in zip(self.names,
                                        candidate_weights.mean(dim=0))
                    }, self.local_step)
            self.local_step += 1
        # batch_size, candidate_vector_dim
        target = torch.bmm(candidate_weights.unsqueeze(dim=1),
                           candidate_vector).squeeze(dim=1)
        return target
    
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask=None):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        scores = torch.exp(scores)
        if attn_mask is not None:
            scores = scores * attn_mask
        attn = scores / (torch.sum(scores, dim=-1, keepdim=True) + 1e-8)

        context = torch.matmul(attn, V)
        return context, attn



class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_attention_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.d_model = d_model
        self.num_attention_heads = num_attention_heads
        assert d_model % num_attention_heads == 0
        self.d_k = d_model // num_attention_heads
        self.d_v = d_model // num_attention_heads

        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)

    def forward(self, Q, K=None, V=None, length=None):
        if K is None:
            K = Q
        if V is None:
            V = Q
        batch_size = Q.size(0)

        q_s = self.W_Q(Q).view(batch_size, -1, self.num_attention_heads,
                               self.d_k).transpose(1, 2)
        k_s = self.W_K(K).view(batch_size, -1, self.num_attention_heads,
                               self.d_k).transpose(1, 2)
        v_s = self.W_V(V).view(batch_size, -1, self.num_attention_heads,
                               self.d_v).transpose(1, 2)

        if length is not None:
            maxlen = Q.size(1)
            attn_mask = torch.arange(maxlen).expand(
                batch_size, maxlen) < length.view(-1, 1)
            attn_mask = attn_mask.unsqueeze(1).expand(batch_size, maxlen,
                                                      maxlen)
            attn_mask = attn_mask.unsqueeze(1).repeat(1,
                                                      self.num_attention_heads,
                                                      1, 1)
        else:
            attn_mask = None

        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s,
                                                            attn_mask)
        context = context.transpose(1, 2).contiguous().view(
            batch_size, -1, self.num_attention_heads * self.d_v)
        return context

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class NewsEncoder(torch.nn.Module):
    def __init__(self, config, word_embedding):
        super(NewsEncoder, self).__init__()
        self.config = config
        self.word_embedding = word_embedding
        self.category_embedding = nn.Embedding(config.num_categories,
                                               config.num_filters,
                                               padding_idx=0)
        assert config.window_size >= 1 and config.window_size % 2 == 1
        
        self.title_CNN = nn.Conv2d(
            1,
            config.num_filters,
            (config.window_size, config.word_embedding_dim),
            padding=(int((config.window_size - 1) / 2), 0))
        self.title_attention = AdditiveAttention(config.news_query_vector_dim,
                                                 config.num_filters)

    def forward(self, news):
        """
        Args:
            news:
                {
                    "category": batch_size,
                    "subcategory": batch_size,
                    "title": batch_size * num_words_title
                }
        Returns:
            (shape) batch_size, num_filters * 3
        """
        #extract title, cat, subcat
        start = self.config.num_words_title 
        title = torch.narrow(news, -1, 0, start).long()
        title_vector = F.dropout(self.word_embedding(title.to(device)),
                                 p=self.config.dropout_probability,
                                 training=self.training)
        convoluted_title_vector = self.title_CNN(title_vector.transpose(1, 2)).transpose(1, 2)
        activated_title_vector = F.dropout(F.relu(convoluted_title_vector),
                                           p=self.config.dropout_probability,
                                           training=self.training)
        weighted_title_vector = self.title_attention(activated_title_vector)
        
        if self.config.use_category:
            category = torch.narrow(news, -1, start, 1).squeeze(dim=-1).long()
            category_vector = self.category_embedding(category.to(device))
            start +=1
        if self.config.use_subcategory:
            subcategory = torch.narrow(news, -1, start, 1).squeeze(dim=-1).long()
            subcategory_vector = self.category_embedding(subcategory.to(device))

        news_vector = torch.cat(
            [weighted_title_vector, category_vector, subcategory_vector],
            dim=1)
        
        return news_vector
    
class UserEncoder(torch.nn.Module):
    def __init__(self, config):
        super(UserEncoder, self).__init__()
        self.config = config
        assert int(config.num_filters * 1.5) == config.num_filters * 1.5
        self.gru = nn.GRU(
            config.num_filters * 3,
            config.num_filters * 3 if config.long_short_term_method == 'ini'
            else int(config.num_filters * 1.5))

    def forward(self, user, clicked_news_length, clicked_news_vector):
        """
        Args:
            user:
                ini: batch_size, num_filters * 3
                con: batch_size, num_filters * 1.5
            clicked_news_length: batch_size,
            clicked_news_vector: batch_size, num_clicked_news_a_user, num_filters * 3
        Returns:
            (shape) batch_size, num_filters * 3
        """

        clicked_news_length = torch.clamp(clicked_news_length, max=clicked_news_vector.shape[1])

        clicked_news_length[clicked_news_length == 0] = 1
        # 1, batch_size, num_filters * 3
        if self.config.long_short_term_method == 'ini':
            packed_clicked_news_vector = pack_padded_sequence(
                clicked_news_vector,
                clicked_news_length,
                batch_first=True,
                enforce_sorted=False)
            _, last_hidden = self.gru(packed_clicked_news_vector,
                                      user.unsqueeze(dim=0))
            return last_hidden.squeeze(dim=0)
        else:
            packed_clicked_news_vector = pack_padded_sequence(
                clicked_news_vector,
                clicked_news_length,
                batch_first=True,
                enforce_sorted=False)
            _, last_hidden = self.gru(packed_clicked_news_vector)
            return torch.cat((last_hidden.squeeze(dim=0), user), dim=1)

class LSTUR(torch.nn.Module):
    """
    LSTUR network.
    Input 1 + K candidate news and a list of user clicked news, produce the click probability.
    """
    def __init__(self, config, embedding_matrix):
        """
        # ini
        user embedding: num_filters * 3
        news encoder: num_filters * 3
        GRU:
        input: num_filters * 3
        hidden: num_filters * 3

        # con
        user embedding: num_filter * 1.5
        news encoder: num_filters * 3
        GRU:
        input: num_fitlers * 3
        hidden: num_filter * 1.5
        """
        super(LSTUR, self).__init__()
        self.config = config
        word_embedding = torch.from_numpy(embedding_matrix).float()
        pretrained_word_embedding = nn.Embedding.from_pretrained(word_embedding,
                                                      freeze=args.freeze_embedding,
                                                      padding_idx=0)
        self.news_encoder = NewsEncoder(config, pretrained_word_embedding)
        self.user_encoder = UserEncoder(config)
        self.click_predictor = DotProductClickPredictor()
        assert int(config.num_filters * 1.5) == config.num_filters * 1.5
        self.user_embedding = nn.Embedding(
            config.num_users,
            config.num_filters * 3 if config.long_short_term_method == 'ini'
            else int(config.num_filters * 1.5),
            padding_idx=0)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, user, clicked_news_length, candidate_news, clicked_news, label):
        """
        Args:
            user: batch_size,
            clicked_news_length: batch_size,
            candidate_news:
                [
                    {
                        "category": batch_size,
                        "subcategory": batch_size,
                        "title": batch_size * num_words_title
                    } * (1 + K)
                ]
            clicked_news:
                [
                    {
                        "category": batch_size,
                        "subcategory": batch_size,
                        "title": batch_size * num_words_title
                    } * num_clicked_news_a_user
                ]
        Returns:
            click_probability: batch_size
        """
        # batch_size, 1 + K, num_filters * 3
        candidate_news_vector = torch.stack(
            [self.news_encoder(x) for x in candidate_news])
        # ini: batch_size, num_filters * 3
        # con: batch_size, num_filters * 1.5
        # TODO what if not drop
        user = F.dropout2d(self.user_embedding(
            user.to(device)).unsqueeze(dim=0),
                           p=self.config.masking_probability,
                           training=self.training).squeeze(dim=0)
        # batch_size, num_clicked_news_a_user, num_filters * 3
        clicked_news_vector = torch.stack(
            [self.news_encoder(x) for x in clicked_news])
        # batch_size, num_filters * 3
        user_vector = self.user_encoder(user, clicked_news_length,
                                        clicked_news_vector)
        # batch_size, 1 + K
        click_probability = self.click_predictor(candidate_news_vector,
                                                 user_vector)
        loss = self.loss_fn(click_probability, label)
        return loss, click_probability

    def get_news_vector(self, news):
        # batch_size, num_filters * 3
        return self.news_encoder(news)

    def get_user_vector(self, user, clicked_news_length, clicked_news_vector):
        """
        Args:
            user: batch_size
            clicked_news_length: batch_size
            clicked_news_vector: batch_size, num_clicked_news_a_user, num_filters * 3
        Returns:
            (shape) batch_size, num_filters * 3
        """
        # ini: batch_size, num_filters * 3
        # con: batch_size, num_filters * 1.5
        user = self.user_embedding(user.to(device))
        # batch_size, num_filters * 3
        return self.user_encoder(user, clicked_news_length,
                                 clicked_news_vector)

    def get_prediction(self, news_vector, user_vector):
        """
        Args:
            news_vector: candidate_size, word_embedding_dim
            user_vector: word_embedding_dim
        Returns:
            click_probability: candidate_size
        """
        # candidate_size
        return self.click_predictor(
            news_vector.unsqueeze(dim=0),
            user_vector.unsqueeze(dim=0)).squeeze(dim=0)

In [12]:
args.mode = 'train'

In [11]:
if 'train' in args.mode:
    if args.prepare:
        logging.info('Preparing training data...')
        total_sample_num = prepare_training_data(args.train_data_dir, args.nGPU, args.npratio, args.seed)
    else:
        total_sample_num = 0
        for i in range(args.nGPU):
            data_file_path = os.path.join(args.train_data_dir, f'behaviors_np{args.npratio}_{i}.tsv')
            print(data_file_path)
            if not os.path.exists(data_file_path):
                logging.error(f'Splited training data {data_file_path} for GPU {i} does not exist. Please set the parameter --prepare as True and rerun the code.')
                exit()
            result = subprocess.getoutput(f'wc -l {data_file_path}')
            total_sample_num += int(result.split(' ')[0])
        logging.info('Skip training data preparation.')
    logging.info(f'{total_sample_num} training samples, {total_sample_num // args.batch_size // args.nGPU} batches in total.')



[INFO 2025-02-25 00:45:29,567] Preparing training data...


156965it [00:02, 68968.77it/s]


[INFO 2025-02-25 00:45:32,033] Writing files...
[INFO 2025-02-25 00:45:32,280] 236344 training samples, 7385 batches in total.


In [12]:
rank = 0
news, news_index, category_dict, subcategory_dict, word_dict = read_news(
		os.path.join(args.train_data_dir, 'news.tsv'), args.train_abstract_dir, args, mode='train')

news_title, news_category, news_subcategory, news_abstract = get_doc_input(
    news, news_index, category_dict, subcategory_dict, word_dict, args)
news_combined = np.concatenate([x for x in [news_title, news_category, news_subcategory] if x is not None], axis=-1)

if rank == 0:
    logging.info('Initializing word embedding matrix...')

embedding_matrix, have_word = load_matrix(args.glove_embedding_path,
                                                word_dict,
                                                args.word_embedding_dim)
if rank == 0:
    logging.info(f'Word dict length: {len(word_dict)}')
    logging.info(f'Have words: {len(have_word)}')
    logging.info(f'Missing rate: {(len(word_dict) - len(have_word)) / len(word_dict)}')

51282it [00:02, 17362.08it/s]
100%|██████████| 51282/51282 [00:00<00:00, 178713.84it/s]

[INFO 2025-02-25 00:45:36,256] Initializing word embedding matrix...


[INFO 2025-02-25 00:46:11,157] Word dict length: 12519
[INFO 2025-02-25 00:46:11,157] Have words: 11960
[INFO 2025-02-25 00:46:11,158] Missing rate: 0.0446521287642783


In [18]:
import torch.optim as optim

model = LSTUR(args, embedding_matrix)
is_distributed = False
'''
if args.load_ckpt_name is not None:
	ckpt_path = get_checkpoint(args.model_dir, args.load_ckpt_name)
	checkpoint = torch.load(ckpt_path, map_location='cpu')
	model.load_state_dict(checkpoint['model_state_dict'])
	logging.info(f"Model loaded from {ckpt_path}.")
'''
optimizer = optim.Adam(model.parameters(), lr=args.lr)

if args.enable_gpu:
	model = model.cuda(rank)

if is_distributed:
	model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[rank])

data_file_path = os.path.join(args.train_data_dir, f'behaviors_np{args.npratio}_{rank}.tsv')

dataset = DatasetTrain(data_file_path, news_index, news_combined, "/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/MINDsmall_train/user2id.json", args)

dataloader = DataLoader(dataset, batch_size=args.batch_size)

logging.info('Training...')
for ep in range(args.start_epoch, args.epochs):
	loss = 0.0
	accuary = 0.0
	for cnt, (user_index, clicked_news_length, candidate_news_feature, clicked_news_feature, label) in enumerate(dataloader):
		if args.enable_gpu:
			user_index = user_index.cuda(rank, non_blocking=True)
			clicked_news_length = clicked_news_length.cuda(rank, non_blocking=True)
			candidate_news_feature = candidate_news_feature.cuda(rank, non_blocking=True)
			clicked_news_feature = clicked_news_feature.cuda(rank, non_blocking=True)
			label = label.cuda(rank, non_blocking=True)
		
		bz_loss, y_hat = model(user_index, clicked_news_length, candidate_news_feature, clicked_news_feature, label)
		loss += bz_loss.data.float()
		accuary += acc(label, y_hat)
		optimizer.zero_grad()
		bz_loss.backward()
		optimizer.step()

		if cnt % args.log_steps == 0:
			logging.info(
				'[{}] Ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(
					rank, cnt * args.batch_size, loss.data / cnt, accuary / cnt)
			)

		if rank == 0 and cnt != 0 and cnt % args.save_steps == 0:
			ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}-{cnt}.pt')
			torch.save(
				{
					'model_state_dict':
						{'.'.join(k.split('.')[1:]): v for k, v in model.state_dict().items()}
						if is_distributed else model.state_dict(),
					'category_dict': category_dict,
					'word_dict': word_dict,
					'subcategory_dict': subcategory_dict
				}, ckpt_path)
			logging.info(f"Model saved to {ckpt_path}.")

	logging.info('Training finish.')

	if rank == 0:
		ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}.pt')
		torch.save(
			{
				'model_state_dict':
					{'.'.join(k.split('.')[1:]): v for k, v in model.state_dict().items()}
					if is_distributed else model.state_dict(),
				'category_dict': category_dict,
				'subcategory_dict': subcategory_dict,
				'word_dict': word_dict,
			}, ckpt_path)
		logging.info(f"Model saved to {ckpt_path}.")



TabError: inconsistent use of tabs and spaces in indentation (<string>, line 38)

In [43]:
args.mode = 'test'
args.user_log_mask=True
args.batch_size=128
args.load_ckpt_name= 'epoch-5.pt'
args.prepare=True
if 'test' in args.mode:
        if args.prepare:
            logging.info('Preparing testing data...')
            total_sample_num = prepare_testing_data(args.test_data_dir, args.nGPU)
        else:
            total_sample_num = 0
            for i in range(args.nGPU):
                data_file_path = os.path.join(args.test_data_dir, f'behaviors_{i}.tsv')
                if not os.path.exists(data_file_path):
                    logging.error(f'Splited testing data {data_file_path} for GPU {i} does not exist. Please set the parameter --prepare as True and rerun the code.')
                    exit()
                result = subprocess.getoutput(f'wc -l {data_file_path}')
                total_sample_num += int(result.split(' ')[0])
            logging.info('Skip testing data preparation.')
        logging.info(f'{total_sample_num} testing samples in total.')

        test(0, args)

[INFO 2025-02-19 15:58:35,169] Preparing testing data...
[INFO 2025-02-19 15:58:35,169] Preparing testing data...
[INFO 2025-02-19 15:58:35,169] Preparing testing data...
[INFO 2025-02-19 15:58:35,169] Preparing testing data...
[INFO 2025-02-19 15:58:35,169] Preparing testing data...


73152it [00:00, 489730.01it/s]

[INFO 2025-02-19 15:58:35,326] Writing files...
[INFO 2025-02-19 15:58:35,326] Writing files...
[INFO 2025-02-19 15:58:35,326] Writing files...
[INFO 2025-02-19 15:58:35,326] Writing files...
[INFO 2025-02-19 15:58:35,326] Writing files...


[INFO 2025-02-19 15:58:35,483] 73152 testing samples in total.
[INFO 2025-02-19 15:58:35,483] 73152 testing samples in total.
[INFO 2025-02-19 15:58:35,483] 73152 testing samples in total.
[INFO 2025-02-19 15:58:35,483] 73152 testing samples in total.
[INFO 2025-02-19 15:58:35,483] 73152 testing samples in total.


NameError: name 'test' is not defined

# Test


In [39]:
label = random.randint(0, 4)
label

2

In [40]:
!mkdir data
!cd data

# Dowload GloVe pre-trained word embedding and unzip
!wget https://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

# Download MIND-small dataset and unzip
!wget https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
!wget https://mind201910small.blob.core.windows.net/release/MINDsmall_dev.zip
!unzip MINDsmall_train.zip -d MINDsmall_train
!unzip MINDsmall_dev.zip -d MINDsmall_dev

!rm glove.840B.300d.zip
!rm MINDsmall_train.zip
!rm MINDsmall_dev.zip

--2025-02-16 04:36:04--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2025-02-16 04:36:04--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  4.99MB/s    in 6m 49s  

2025-02-16 04:42:54 (5.07 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     
--2